In [1]:
!pip install torch_geometric

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 348kB 8.5MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.4.3-cp36-none-any.whl size=234873 sha256=e23534268ff14822aad1aea1d78ebe7ef2940257705f3c24bc81185a11d200d2
  Stored in directory: /root/.cache/pip/wheels/e2/c1/09/8693feee3f97e440d68b09abfca8b4c1e97150ace350b5003f
  Created wheel for plyfile: filename=plyfile-0.7.1-cp36-none-any.whl size=32828 sha256=a9a1c415bb28cd1883b7c06f469ef03e0e58ecf3a4a7847bcf80602d0d56b2a5
  Stored in directory: /root/.cache/pip/wheels/d6/0d/bf/6d603d81b98604d2ecfd5e99d4ab7c9af664fd5285ab82bbb0
Successfully built torch-geometric plyfile


In [2]:
!pip install torch-scatter==latest+cu101 torch-sparse==latest+cu101 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html

Looking in links: https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html
     |████████████████████████████████| 10.6MB 464kB/s 
     |████████████████████████████████| 11.5MB 747kB/s 


In [0]:
import sys
sys.path.insert(0, '/content/drive/My Drive')

In [0]:
import torch
import numpy as np
import networkx as nx
import torch.nn as nn
import time
import torch.nn.functional as F

In [0]:
from utils import EarlyStopping

In [0]:
from torch_geometric.datasets import Planetoid

In [0]:
Cora = Planetoid(root='/content/drive/My Drive/', name='Cora')
cora = Cora[0]

In [0]:
C_S = Planetoid(root='/content/drive/My Drive/', name='CiteSeer')
cite_seer = C_S[0]

In [0]:
PM = Planetoid(root='/content/drive/My Drive/', name='PubMed')
pub_med = PM[0]

In [0]:
# for compress test
cora = torch.load('/content/drive/My Drive/cora_193.pt')

In [0]:
data = cora
#data.x = cora.x[:, :800] 
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
num_features

193

## D_inv*A

In [0]:
from D_inv_A import D_inv_AConv

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = D_inv_AConv(
            num_features, num_classes, K=3, cached=True, improve= True)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        return F.log_softmax(x, dim=1)

In [0]:
das = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  #cora_da
  optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-3)
  #pubmed_da
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-4)
  #cite_seer_da
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-1)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  das.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): D_inv_AConv(193, 7, K=3)
)

Test Accuracy 0.7920
Net(
  (conv1): D_inv_AConv(193, 7, K=3)
)

Test Accuracy 0.7900
Net(
  (conv1): D_inv_AConv(193, 7, K=3)
)

Test Accuracy 0.7920
Net(
  (conv1): D_inv_AConv(193, 7, K=3)
)

Test Accuracy 0.7970
Net(
  (conv1): D_inv_AConv(193, 7, K=3)
)

Test Accuracy 0.7930
Net(
  (conv1): D_inv_AConv(193, 7, K=3)
)

Test Accuracy 0.7800
Net(
  (conv1): D_inv_AConv(193, 7, K=3)
)

Test Accuracy 0.7940
Net(
  (conv1): D_inv_AConv(193, 7, K=3)
)

Test Accuracy 0.7910
Net(
  (conv1): D_inv_AConv(193, 7, K=3)
)

Test Accuracy 0.7940
Net(
  (conv1): D_inv_AConv(193, 7, K=3)
)

Test Accuracy 0.7960


In [0]:
das

[0.792, 0.79, 0.792, 0.797, 0.793, 0.78, 0.794, 0.791, 0.794, 0.796]

In [0]:
np.mean(das)

0.7919000000000002

## D_A_D

In [0]:
from D_A_D import P_GCNConv

In [0]:
data = pub_med
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = P_GCNConv(
            num_features, num_classes, K=3, cached=True, improve=True)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        return F.log_softmax(x, dim=1)

In [0]:
dads = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  #cora_dad
  optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-3)
  #pubmed_dad
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-4)
  #cite_seer_dad
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-2)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() # 空出一行
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  dads.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): P_GCNConv(193, 7, K=3)
)

Test Accuracy 0.7940
Net(
  (conv1): P_GCNConv(193, 7, K=3)
)

Test Accuracy 0.7780
Net(
  (conv1): P_GCNConv(193, 7, K=3)
)

Test Accuracy 0.7970
Net(
  (conv1): P_GCNConv(193, 7, K=3)
)

Test Accuracy 0.7930
Net(
  (conv1): P_GCNConv(193, 7, K=3)
)

Test Accuracy 0.7950
Net(
  (conv1): P_GCNConv(193, 7, K=3)
)

Test Accuracy 0.7930
Net(
  (conv1): P_GCNConv(193, 7, K=3)
)

Test Accuracy 0.7830
Net(
  (conv1): P_GCNConv(193, 7, K=3)
)

Test Accuracy 0.7840
Net(
  (conv1): P_GCNConv(193, 7, K=3)
)

Test Accuracy 0.7880
Net(
  (conv1): P_GCNConv(193, 7, K=3)
)

Test Accuracy 0.7940


In [0]:
dads

[0.794, 0.778, 0.797, 0.793, 0.795, 0.793, 0.783, 0.784, 0.788, 0.794]

In [0]:
np.mean(dads)

0.7899

## P-AGNN

In [0]:
from cos_conv import P_AGNNConv

In [0]:
data = pub_med
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = P_AGNNConv(
            num_features, num_classes, K=3,beta=1.0, cached=True, improved= True)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        return F.log_softmax(x, dim=1)

In [0]:
agnns = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  #cora_agnn
  optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-3)
  #pubmed_agnn
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-4)
  #cite_seer_agnn
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=5e-1)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  agnns.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): P_AGNNConv(193, 7, K=3)
)

Test Accuracy 0.8000
Net(
  (conv1): P_AGNNConv(193, 7, K=3)
)

Test Accuracy 0.7820
Net(
  (conv1): P_AGNNConv(193, 7, K=3)
)

Test Accuracy 0.7980
Net(
  (conv1): P_AGNNConv(193, 7, K=3)
)

Test Accuracy 0.7890
Net(
  (conv1): P_AGNNConv(193, 7, K=3)
)

Test Accuracy 0.7910
Net(
  (conv1): P_AGNNConv(193, 7, K=3)
)

Test Accuracy 0.7930
Net(
  (conv1): P_AGNNConv(193, 7, K=3)
)

Test Accuracy 0.7890
Net(
  (conv1): P_AGNNConv(193, 7, K=3)
)

Test Accuracy 0.7920
Net(
  (conv1): P_AGNNConv(193, 7, K=3)
)

Test Accuracy 0.7860
Net(
  (conv1): P_AGNNConv(193, 7, K=3)
)

Test Accuracy 0.7970


In [0]:
agnns

[0.8, 0.782, 0.798, 0.789, 0.791, 0.793, 0.789, 0.792, 0.786, 0.797]

In [0]:
np.mean(agnns)

0.7917000000000001

## P-GAT

In [0]:
import torch
import torch.nn as nn
from torch.nn import Linear
from torch.nn import Parameter
import torch.nn.functional as F
from torch_geometric.nn.inits import glorot, zeros
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax


class P_GATConv(MessagePassing):

    def __init__(self, in_channels, out_channels, k=2, negative_slope=0.2, dropout=0.6, bias=True, improved=False,drop=False,**kwargs):
        super(P_GATConv, self).__init__(aggr='add', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.k = k
        self.negative_slope = negative_slope
        self.dropout = dropout
        self.bias = bias
        self.improved = improved
        self.drop = drop

        self.lin = Linear(in_channels, out_channels, bias=bias)
        self.att = Parameter(torch.Tensor(1, 2 * out_channels))
      
        self.reset_parameters()

    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_normal_(self.lin.weight, gain=gain)
        nn.init.xavier_normal_(self.att, gain=gain)
        if self.bias:
            nn.init.zeros_(self.lin.bias)

    ''' def reset_parameters(self):
        self.lin.reset_parameters()
        #self.lin2.reset_parameters()
        glorot(self.att)'''
    
    def My_norms(self, x, edge_index, num_nodes, improved=False, drop=False):
        edge_index_j = edge_index[0]
        edge_index_i = edge_index[1]
        x_j = x[edge_index_j]
        x_i = x[edge_index_i]
        
        '''alpha0 = (torch.cat([x_i, x_j], dim=-1) * self.att).sum(dim=-1)
        alpha1 = (torch.cat([x_j, x_i], dim=-1) * self.att).sum(dim=-1)
        alpha = (alpha0 + alpha1)/2.0'''

        if improved:
            self_weight = edge_index_i== edge_index_j
            alpha[self_weight] += 1.0
        
        alpha = (torch.cat([x_i, x_j], dim=-1) * self.att).sum(dim=-1)

        alpha = F.leaky_relu(alpha, self.negative_slope)

        
        alpha = softmax(alpha, edge_index_i, num_nodes)
        
        # Sample attention coefficients stochastically.
        if self.drop:
            alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        
        return alpha
    
    def forward(self, x, edge_index, size=None):
        """"""
        if size is None and torch.is_tensor(x):
            edge_index, _ = remove_self_loops(edge_index)
            edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        x = self.lin(x)
        
        norm = self.My_norms(x, edge_index, x.size(0))
            
        for _ in range(self.k):

            x = self.propagate(edge_index, size=size, x=x, norm=norm)
        
        #x = self.lin(x)
        return x


    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j
    
    def __repr__(self):
        return '{}({}, {}, k={})'.format(self.__class__.__name__,
                                             self.in_channels,
                                             self.out_channels, self.k)

In [0]:
from p_gat import P_GATConv

In [0]:
data = pub_med
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [51]:
data

Data(edge_index=[2, 88648], test_mask=[19717], train_mask=[19717], val_mask=[19717], x=[19717, 500], y=[19717])

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = P_GATConv(
            num_features, num_classes, k=3, improved=True, drop=True)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = self.conv(x, edge_index)
        return F.log_softmax(x, dim=1)

In [58]:
gats = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  #cora_gat
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-2)
  #pubmed_gat
  optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-4)
  #cite_seer_gat
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-2)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print()
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  gats.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv): P_GATConv(500, 3, k=3)
)

Test Accuracy 0.7570
Net(
  (conv): P_GATConv(500, 3, k=3)
)

Test Accuracy 0.7720
Net(
  (conv): P_GATConv(500, 3, k=3)
)

Test Accuracy 0.7860
Net(
  (conv): P_GATConv(500, 3, k=3)
)

Test Accuracy 0.7710
Net(
  (conv): P_GATConv(500, 3, k=3)
)

Test Accuracy 0.7670
Net(
  (conv): P_GATConv(500, 3, k=3)
)

Test Accuracy 0.7820
Net(
  (conv): P_GATConv(500, 3, k=3)
)

Test Accuracy 0.7810
Net(
  (conv): P_GATConv(500, 3, k=3)
)

Test Accuracy 0.7640
Net(
  (conv): P_GATConv(500, 3, k=3)
)

Test Accuracy 0.7680
Net(
  (conv): P_GATConv(500, 3, k=3)
)

Test Accuracy 0.7830


In [59]:
print('done2')

done2


In [60]:
gats

[0.757, 0.772, 0.786, 0.771, 0.767, 0.782, 0.781, 0.764, 0.768, 0.783]

In [61]:
np.mean(gats)*100

77.31

In [62]:
np.std(gats)*100

0.9038252043398665